In [1]:
from dask_gateway import Gateway
from dask_gateway import GatewayCluster
from dask.distributed import Client


In [2]:
gateway = Gateway()

In [3]:
gateway

Gateway<http://10.100.222.95:8000/services/dask-gateway>

In [4]:
options = gateway.cluster_options(); options

In [5]:
cluster = gateway.new_cluster(); cluster

In [6]:
clusters = gateway.list_clusters(); clusters

[ClusterReport<name=daskhub.3c7801df4d2f465e9187166bfad9e2b4, status=RUNNING>]

In [7]:
gateway.list_clusters()[0].name

'daskhub.3c7801df4d2f465e9187166bfad9e2b4'

In [8]:
client = cluster.get_client();

distributed.comm.tcp - WARNING - Closing dangling stream in <TLS  local=tls://192.168.20.40:48986 remote=gateway://traefik-dhub-dask-gateway.daskhub:80/daskhub.3c7801df4d2f465e9187166bfad9e2b4>


OSError: Timed out trying to connect to 'gateway://traefik-dhub-dask-gateway.daskhub:80/daskhub.3c7801df4d2f465e9187166bfad9e2b4' after 10 s: Timed out trying to connect to 'gateway://traefik-dhub-dask-gateway.daskhub:80/daskhub.3c7801df4d2f465e9187166bfad9e2b4' after 10 s: connect() didn't finish in time

In [9]:
client

NameError: name 'client' is not defined

In [10]:
import dask.array as da
a = da.random.normal(size=(1000,1000),chunks=(50,50))
a.mean().compute()

-5.581990759918615e-05

In [11]:
client.nthreads()

NameError: name 'client' is not defined

In [12]:
cluster.shutdown()

In [13]:
cluster.close()

In [14]:
gateway.list_clusters()

[]

In [ ]:
cluster = gateway.connect(clusters[0].name)

In [ ]:
cluster.shutdown(); cluster.close()

In [16]:
gateway.close()

In [17]:
gateway.list_clusters()

[]

In [18]:
gateway

Gateway<http://10.100.222.95:8000/services/dask-gateway>